### Import

In [0]:
#!pip install sklearn_pandas

In [0]:
def temp_to_min(X):
    hour, minute = pd.Series(X).str.split(" ", 1).str
    hour = pd.Series(hour).str.replace("h", " ")
    hour = hour.astype('int64')
    minute = pd.Series(minute).str.replace("m", " ")
    minute = minute.fillna(0)
    minute = minute.astype('int64')
    total_minute = pd.to_timedelta( (hour * 60) + minute, unit = "m" )
    return(total_minute)

### Load modules

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn_pandas import DataFrameMapper, FunctionTransformer, CategoricalImputer
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [0]:
from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Data Cleaning

In [0]:
train_data = pd.read_csv("drive/My Drive/PyColab Work/FlightPricePredict/train.csv")
sub_data = pd.read_csv("drive/My Drive/PyColab Work/FlightPricePredict/test.csv")
#train_data.drop(index = 6474, inplace = True, axis = 0) # 5m flight time #change to 24hr 5m
train_data.iloc[6474, 7] = "24h 5m"
sub_data.iloc[2660, 7] = "24h 5m"

train_data.iloc[1478, 0] = "Jet Airways Business"
train_data.iloc[2618, 0] = "Jet Airways Business"
train_data.iloc[5439, 0] = "Jet Airways Business"

train_data.drop(index = 2924, inplace = True, axis = 0)
#train_data.drop(index = 5136, inplace = True, axis = 0)
#train_data.drop(index = 2618, inplace = True, axis = 0)
#train_data.drop(index = 5439, inplace = True, axis = 0)


# Date need to be looked at # Move this
train_data["Week"] = pd.to_datetime(train_data["Date_of_Journey"], dayfirst = True).dt.weekday_name
train_data["Month"] = pd.to_datetime(train_data["Date_of_Journey"], dayfirst = True).dt.month
train_data["Airline"] = train_data.Airline.replace("Trujet", "SpiceJet")
train_data["class"] = np.where(train_data['Airline']=='Jet Airways Business', 'High', 'Low')
train_data["meal"] = np.where(train_data['Additional_Info']=='In-flight meal not included', 'High', 'Low')
train_data["checkin"] = np.where(train_data['Additional_Info']=='No check-in baggage included', 'High', 'Low')
train_data["Destination"] = np.where(train_data["Destination"] == "New Delhi", "Delhi", train_data["Destination"])
train_data["Airline"] = np.where(train_data["Airline"] == "Vistara Premium economy", "Multiple carriers", train_data["Airline"])
train_data["Airline"] = np.where(train_data["Airline"] == "Multiple carriers Premium economy", "Multiple carriers", train_data["Airline"])
#train_data["S1"], train_data["S2"], train_data['S3'], train_data["S4"], train_data['S5'], train_data["S6"] = train_data.Route.str.replace("?", "-").str.split(" - ").str
#train_data.fillna("NA", axis = 1, inplace = True)
train_data['landing'] = pd.to_datetime(train_data['Date_of_Journey'] + '-' + train_data['Dep_Time'], dayfirst = True) + temp_to_min(train_data["Duration"])
train_data["landingWeek"] = pd.to_datetime(train_data["landing"]).dt.weekday_name
train_data["Source"] = train_data["Source"] + train_data["Destination"]

  
train_data['landing'] = pd.to_datetime(train_data['Date_of_Journey'] + '-' + train_data['Dep_Time']) + temp_to_min(train_data["Duration"])
train_data["landingWeek"] = pd.to_datetime(train_data["landing"]).dt.weekday_name
train_data.drop(['Route', 'Destination'], axis = 1).head()

#Seperate y here itself

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Week,Month,class,meal,checkin,landing,landingWeek
0,IndiGo,24/03/2019,BangloreDelhi,Delhi,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,Sunday,3,Low,Low,Low,2019-03-25 01:10:00,Monday
1,Air India,1/05/2019,KolkataBanglore,Banglore,05:50,13:15,7h 25m,2 stops,No info,7662,Wednesday,5,Low,Low,Low,2019-01-05 13:15:00,Saturday
2,Jet Airways,9/06/2019,DelhiCochin,Cochin,09:25,04:25 10 Jun,19h,2 stops,No info,13882,Sunday,6,Low,Low,Low,2019-09-07 04:25:00,Saturday
3,IndiGo,12/05/2019,KolkataBanglore,Banglore,18:05,23:30,5h 25m,1 stop,No info,6218,Sunday,5,Low,Low,Low,2019-12-05 23:30:00,Thursday
4,IndiGo,01/03/2019,BangloreDelhi,Delhi,16:50,21:35,4h 45m,1 stop,No info,13302,Friday,3,Low,Low,Low,2019-01-03 21:35:00,Thursday


In [0]:
train_data.Airline.value_counts()

Jet Airways             3846
IndiGo                  2053
Air India               1752
Multiple carriers       1212
SpiceJet                 819
Vistara                  479
Air Asia                 319
GoAir                    194
Jet Airways Business       8
Name: Airline, dtype: int64

In [0]:
%%script false

def flight_data_clean(df) :
  df["Week"] = pd.to_datetime(df["Date_of_Journey"]).dt.weekday
  df["Month"] = pd.to_datetime(df["Date_of_Journey"]).dt.month
  df = df.drop(['Date_of_Journey','Route', 'Additional_Info'], axis = 1)
  df = pd.get_dummies(df, columns=['Airline', 'Source', 'Destination', 'Total_Stops'])
  return(df)

flight_data_clean(train_data)

In [0]:
sub_data["Week"] = pd.to_datetime(sub_data["Date_of_Journey"], dayfirst = True).dt.weekday_name
sub_data["Month"] = pd.to_datetime(sub_data["Date_of_Journey"], dayfirst = True).dt.month
sub_data["class"] = np.where(sub_data['Airline']=='Jet Airways Business', 'High', 'Low')
sub_data["meal"] = np.where(sub_data['Additional_Info']=='In-flight meal not included', 'High', 'Low')
sub_data["checkin"] = np.where(sub_data['Additional_Info']=='No check-in baggage included', 'High', 'Low')
sub_data["Destination"] = np.where(sub_data["Destination"] == "New Delhi", "Delhi", sub_data["Destination"])
train_data["Airline"] = np.where(train_data["Airline"] == "Vistara Premium economy", "Multiple carriers", train_data["Airline"])
train_data["Airline"] = np.where(train_data["Airline"] == "Multiple carriers Premium economy", "Multiple carriers", train_data["Airline"])
sub_data['landing'] = pd.to_datetime(sub_data['Date_of_Journey'] + '-' + sub_data['Dep_Time']) + temp_to_min(sub_data["Duration"])
sub_data["landingWeek"] = pd.to_datetime(sub_data["landing"]).dt.weekday_name
#sub_data["S1"], sub_data["S2"], sub_data['S3'], sub_data["S4"], sub_data['S5'], sub_data["S6"] = sub_data.Route.str.replace("?", "-").str.split(" - ").str
#sub_data.fillna("NA", axis = 1, inplace = True)
train_data['landing'] = pd.to_datetime(train_data['Date_of_Journey'] + '-' + train_data['Dep_Time'], dayfirst = True) + temp_to_min(train_data["Duration"])
train_data["landingWeek"] = pd.to_datetime(train_data["landing"]).dt.weekday_name
train_data["Source"] = train_data["Source"] + train_data["Destination"]

In [0]:
encoder = LabelBinarizer() # LabelBinarizer()  ~~#LabelEncoder()~~
to_bin = KBinsDiscretizer(n_bins=3, encode = 'onehot-dense')


class to_min(TransformerMixin):
  
  def fit(self, X, y=None):
    return self
  
  def transform(self, X):
    hour, minute = pd.Series(X).str.split(" ", 1).str
    hour = pd.Series(hour).str.replace("h", " ")
    hour = hour.astype('int64')
    minute = pd.Series(minute).str.replace("m", " ")
    minute = minute.fillna(0)
    minute = minute.astype('int64')
    total_minute = pd.Series( (hour * 60) + minute )
    #short_dist = pd.Series( np.where(total_minute < 360, 1, 0) )
    #long_dist = pd.Series( np.where(total_minute < 360, 0, 1) )
    #return (pd.concat([total_minute, short_dist, long_dist], axis = 1))
    total_minute = pd.DataFrame(to_bin.fit_transform(pd.DataFrame(total_minute)))
    return(total_minute)

# Can we use subclass?
class part_of_day(TransformerMixin):
  def fit(self, X, y = None):
    return self
  
  def transform(self, X):
    col = pd.Series(X) 
    aseries = col.replace(":\d\d.*", "", regex = True).astype('int32')
    conditions = [
      (aseries > 6) & (aseries <= 13),
      (aseries > 13) & (aseries <= 18),
      (aseries > 18) & (aseries <= 21),
      (aseries > 21) | (aseries <= 6)
    ]

    choices = ['morning', 'afternoon', 'eve', 'night']
  
    return np.select(conditions, choices, default='night')
  

class convert_stops(TransformerMixin):
  def fit(self, X, y = None):
    return self
  
  def transform(self, X):
    col = pd.Series(X) 
    #aseries = 
    conditions = [
      (col == "non-stop"),
      (col == "1 stop"),
      (col == "2 stops"),
      (col == "3 stops"),
      (col == "4 stops")
    ]

    choices = [0, 1, 2, 3, 4]
  
    return np.select(conditions, choices, default='0')

In [0]:

# As Route and Additional_Info are not defined they are dropped. 
# Write a default function for imputation

mapper = DataFrameMapper([
    ( 'Airline', encoder ),
    ( 'Week', encoder ),
    ( 'Month', encoder ),
    ( 'Source', encoder ),
    ( 'Destination', encoder ),
    ( 'Dep_Time', [part_of_day(), encoder] ),
    ( 'Arrival_Time', [part_of_day(), encoder] ),
    #( 'Duration', to_min() ),
    ( 'Total_Stops', [CategoricalImputer(strategy = "most_frequent"), convert_stops()] ),
    ( 'class', encoder),
    ( 'meal', encoder),
    #( 'S1', encoder),
    #( 'S2', encoder),
    #( 'S3', encoder),
    #( 'S4', encoder),
    #( 'S5', encoder),
    ( 'landingWeek', encoder),
    ( 'checkin', encoder) # As these columns are binary, encoder does not apply on them.
    #( 'Price', FunctionTransformer(np.log) ),
], df_out = True
)

In [0]:
X = train_data.drop("Price", axis = 1)
y = np.log(train_data['Price']).astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify = X['Airline']) #Stratify

X_test, X_ftest, y_test, y_ftest = train_test_split(X_test, y_test, test_size = 0.10, stratify = X_test['Airline'])

In [0]:
## IF ERROR THEN RUN PREVIOUS CELL.

# #Prepare pipeline and fit there.
X_train = mapper.fit_transform(X_train).astype(float).drop(['Airline_Jet Airways Business'], axis = 1)
X_test = mapper.fit_transform(X_test).astype(float).drop(['Airline_Jet Airways Business'], axis = 1)

#Can we drop 'Airline_Multiple carriers Premium economy' in onehotencoder and use stacking?
X_ftest = mapper.fit_transform(X_ftest).astype(float)
sub = mapper.fit_transform(sub_data).astype(float).drop(['Airline_Jet Airways Business'], axis = 1)

In [0]:
X_train.columns

Index(['Airline_Air Asia', 'Airline_Air India', 'Airline_GoAir',
       'Airline_IndiGo', 'Airline_Jet Airways', 'Airline_Multiple carriers',
       'Airline_SpiceJet', 'Airline_Vistara', 'Week_Friday', 'Week_Monday',
       'Week_Saturday', 'Week_Sunday', 'Week_Thursday', 'Week_Tuesday',
       'Week_Wednesday', 'Month_3', 'Month_4', 'Month_5', 'Month_6',
       'Source_BangloreDelhiDelhi', 'Source_ChennaiKolkataKolkata',
       'Source_DelhiCochinCochin', 'Source_KolkataBangloreBanglore',
       'Source_MumbaiHyderabadHyderabad', 'Destination_Banglore',
       'Destination_Cochin', 'Destination_Delhi', 'Destination_Hyderabad',
       'Destination_Kolkata', 'Dep_Time_afternoon', 'Dep_Time_eve',
       'Dep_Time_morning', 'Dep_Time_night', 'Arrival_Time_afternoon',
       'Arrival_Time_eve', 'Arrival_Time_morning', 'Arrival_Time_night',
       'Total_Stops', 'class', 'meal', 'landingWeek_Friday',
       'landingWeek_Monday', 'landingWeek_Saturday', 'landingWeek_Sunday',
       'landing

## Boosting

Boosting in following format  
Model based on default values  
Model result on 5 parts of train data, using cv  
RMSE on test data

Hyperparameter tuning with GridSearch cv  
grid result on 5 parts of train data  
RMSE of test data using grid  

Define boosting function

### XGBoost

In [0]:
base_model = xgb.XGBRegressor(learning_rate=0.2, gamma= 0.01, max_depth = 7)
base_model.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(base_model, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
#np.sqrt(mean_squared_error(y_test, base_model.predict(X_test)))

[0.17301387 0.18327945 0.17090015 0.18431581 0.18509788]


In [0]:
xgb_engine = xgb.XGBRegressor() #n_estimator not used
xgb_params = {'max_depth' : np.arange(1, 10, 3), 'gamma' : [0.01, 1], 'learning_rate' : [0.1, 0.2, 0.3]}

xgb_grid = GridSearchCV(xgb_engine, xgb_params, cv = 5, n_jobs = -1, verbose = 1)
xgb_grid.fit(X_train, y_train)

print( np.sqrt( -cross_val_score(xgb_grid.best_estimator_, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
#print( np.sqrt( mean_squared_error(y_test, xgb_grid.predict(X_test)) ) )

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   37.3s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.2min finished


[0.16792932 0.18366188 0.17221405 0.18038188 0.18177267]


### GBM

In [0]:
gbm_model = GradientBoostingRegressor(learning_rate = 0.05, max_depth = 7, n_estimators = 300)
gbm_model.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(gbm_model, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
#np.sqrt(mean_squared_error(y_test, gbm_model.predict(X_test)))

[0.17133946 0.18057683 0.17254508 0.18346578 0.18350606]


In [0]:
gbm_engine = GradientBoostingRegressor()
gbm_params = {'max_depth' : [7], 'learning_rate' : [0.05], 'n_estimators' : [300]}

gbm_grid = GridSearchCV(gbm_engine, gbm_params, cv = 5, n_jobs = -1, verbose = 1)
gbm_grid.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(gbm_grid.best_estimator_, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
#print( np.sqrt( mean_squared_error(y_test, gbm_grid.predict(X_test)) ) )

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   44.8s finished


[0.17139026 0.18038553 0.17257603 0.18338398 0.18363047]


### Light GBM

In [0]:
# https://lightgbm.readthedocs.io/en/latest/Python-API.html#scikit-learn-api
# Use gridsearchcv

import lightgbm as lgb
lgbm_model = lgb.LGBMRegressor(learning_rate = 0.3, n_estimators = 300, max_depth = 6, min_child_samples  = 17)
lgbm_model.fit(X_train.astype(float), y_train.astype(float))

print( np.sqrt( -cross_val_score(lgbm_model, X_train.astype(float), y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
np.sqrt(mean_squared_error(y_test, lgbm_model.predict(X_test)))

[0.17592686 0.1912921  0.17828105 0.18802441 0.18673964]


0.18826264302088913

In [0]:
lgbm_engine = lgb.LGBMRegressor()
lgbm_params = {'max_depth' : [6], 'learning_rate' : [ 0.3], 'n_estimators': [300], 'reg_alpha' : [0],  'min_child_samples' : [17]}

lgbm_grid = GridSearchCV(lgbm_engine, lgbm_params, cv = 5, n_jobs = -1, verbose = 0)
lgbm_grid.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(lgbm_grid.best_estimator_, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
print( np.sqrt( mean_squared_error(y_test, lgbm_grid.predict(X_test)) ) )

[0.17592686 0.1912921  0.17828105 0.18802441 0.18673964]
0.18826264302088913


In [0]:
for a, b, c in zip( lgbm_grid.cv_results_['params'], lgbm_grid.cv_results_['mean_test_score'], lgbm_grid.cv_results_['mean_train_score'] ):
  print(a, b, c)

### AdaBoost

In [0]:
from sklearn.ensemble import AdaBoostRegressor

adb_model = AdaBoostRegressor(n_estimators = 300, learning_rate = 0.2)
adb_model.fit(X_train, y_train)

print( np.sqrt( -cross_val_score(adb_model, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
np.sqrt(mean_squared_error(y_test, adb_model.predict(X_test)))

[0.28595187 0.29305241 0.29082044 0.29939831 0.29153977]


0.29668692877959774

In [0]:
adb_engine = AdaBoostRegressor()
adb_params = {'n_estimators' : np.arange(50, 500, 50), 'learning_rate' : [0.1, 0.2, 0.3]}

adb_grid = GridSearchCV(adb_engine, adb_params, cv = 5, n_jobs = -1, verbose = 1)
adb_grid.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(adb_grid.best_estimator_, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
print( np.sqrt( mean_squared_error(y_test, adb_grid.predict(X_test)) ) )

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  2.1min finished


[0.27507794 0.28970839 0.28506227 0.29452236 0.28188844]
0.29627741886327746


## Bagging

### Random Forest

In [0]:
from sklearn.ensemble import RandomForestRegressor

ranger_model = RandomForestRegressor(n_estimators = 100, oob_score = True)
ranger_model.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(ranger_model, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
np.sqrt(mean_squared_error(y_test, ranger_model.predict(X_test)))

[0.1813913  0.18967487 0.18195193 0.19057174 0.19058201]


0.1932257868916806

In [0]:
ranger_engine = RandomForestRegressor()
ranger_params = {'n_estimators' : np.arange(50, 500, 50)}

ranger_grid = GridSearchCV(ranger_engine, ranger_params, cv = 5, n_jobs = -1, verbose = 1)
ranger_grid.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(ranger_grid.best_estimator_, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
print( np.sqrt( mean_squared_error(y_test, ranger_grid.predict(X_test)) ) )

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  3.4min finished


[0.18084762 0.18929711 0.18173912 0.18973475 0.1901538 ]
0.19226157058661883


## Ensemble

### Average ensemble

In [0]:
# Adaboost overfitting
outcome = pd.DataFrame( {'-actual' : np.exp(y_test) , 
               'a-xgb' :np.exp(xgb_grid.predict(X_test)),
               'b-gbm' : np.exp(gbm_grid.predict(X_test)),
               'c-lgbm' : np.exp(lgbm_grid.predict(X_test))
                        } )
# 'd-ranger' : np.exp(ranger_grid.predict(X_test)) 
outcome['avg'] = outcome.mean(axis = 1)
outcome.describe()

,-actual,a-xgb,b-gbm,c-lgbm,avg
count,2137.000000,2137.000000,2137.000000,2137.000000,2137.000000
mean,9078.092653,8926.119141,8925.628689,8933.927488,8965.942939
std,4566.916347,4186.385254,4234.723963,4119.665644,4201.630358
min,1759.000000,1943.064209,1866.598110,1866.934766,1941.154102
25%,5380.000000,5519.902832,5581.528634,5492.281855,5571.275555
50%,8330.000000,8816.053711,8882.584493,8821.118967,8866.403190
75%,12384.000000,11671.946289,11641.466018,11705.730757,11793.564616
max,57209.000000,58919.238281,56854.062812,35158.803572,50279.161079


In [0]:
# ens_predict = pd.DataFrame( { 'axgbpre' :np.exp(base_model.predict(X_test)) ,
#                'brfpre' : np.exp(ranger.predict(X_test)) } ).mean(axis = 1)

ens_predict = outcome['avg']

np.sqrt( mean_squared_error(np.exp(y_test), ens_predict) )

1222.755684236997

In [0]:
np.sqrt( mean_squared_error(np.exp(y_test), ens_predict) ) - np.sqrt( mean_squared_error(y_test, np.exp(ens_predict)) )

Submit file

In [0]:
#%%script false

soutcome = pd.DataFrame({'a-xgb' :np.exp(xgb_grid.predict(sub)),
               'b-gbm' : np.exp(gbm_grid.predict(sub)),
               'c-lgbm' : np.exp(lgbm_grid.predict(sub.astype(float))) } )

soutcome['avg'] = soutcome.mean(axis = 1)

pd.DataFrame({ "Price" : soutcome['avg'] }, index = None).to_csv("FlightPriceEn.csv")

### Stacking

A pipeline needed

In [0]:
#%%script false

X_stack = pd.DataFrame( {'axgbpre' :xgb_grid.predict(X_test) ,
                         'gbm' : gbm_grid.predict(X_test),
                         'lgbm' : lgbm_grid.predict(X_test) })

y_stack = y_test

In [0]:
#%%script false

stack_model = xgb.XGBRegressor(learning_rate=0.05, n_estimators = 1000) #learning_rate=0.2, gamma= 0.01, max_depth = 7
#stack_model.fit(X_stack, y_stack)
params = {"max_depth" : [2, 5, 7]}
ensemble_grid = GridSearchCV(stack_model, params, cv = 5)
ensemble_grid.fit(X_stack, y_stack)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [2, 5, 7]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [0]:
#%%script false

ensemble_grid.best_estimator_

In [0]:
#%%script false

np.sqrt( -cross_val_score(ensemble_grid, X_stack, y_stack, cv = 5, scoring = "neg_mean_squared_error") )

array([0.1929779 , 0.20005908, 0.17393749, 0.19190841, 0.19283922])

In [0]:
#%%script false

#column Airline_Jet Airways Business, Airline_Multiple carriers Premium economy are not available in ftest
X_stack = pd.DataFrame( {'axgbpre' :xgb_grid.predict(X_ftest) ,
                         'gbm' : gbm_grid.predict(X_ftest),
                         'lgbm' : lgbm_grid.predict(X_ftest) })

y_stack = y_ftest
np.sqrt( mean_squared_error(np.exp(y_ftest), np.exp(ensemble_grid.predict(X_stack))) )

16848.24396010376

## KNN

In [0]:
from sklearn.neighbors import KNeighborsRegressor

## Playground models

### Support Vector Regression - not amazing
Apply appropriate transformation

In [0]:
%%script false

from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, normalize

#scaler = StandardScaler()
#svr_model = SVR(gamma = 'scale')

svmpipe = Pipeline([ ( "std", StandardScaler() ), ( "svm", SVR(gamma = 'scale', kernel = 'rbf', C = 1.5, epsilon = 0.1, max_iter=-1) ) ])
svmpipe.fit(X_train.astype(float), y_train.astype(float))

np.sqrt( -cross_val_score(svmpipe, X_train.astype(float), y_train.astype(float), cv = 5, scoring = "neg_mean_squared_error") )
np.sqrt( mean_squared_error(np.exp(y_test), np.exp(svmpipe.predict(X_test.astype(float)))) )
pd.DataFrame( {'act' : np.exp(y_test) , 'axgbpre' :np.exp(svmpipe.predict(X_test.astype(float)))  } ).describe()

### Ridge - not a good result

In [0]:
%%script false

from sklearn.linear_model import RidgeCV

ridge_model = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1], cv = 5).fit(X_train.astype(float), y_train.astype(float))
np.sqrt( -cross_val_score(ridge_model, X_train.astype(float), y_train.astype(float), cv = 5, scoring = "neg_mean_squared_error") )
np.sqrt(mean_squared_error(npa.exp(y_test.astype(float)), np.exp(ridge_model.predict(X_test.astype(float)))))

pd.DataFrame( {'act' : np.exp(y_test) , 
               'axgbpre' :np.exp(base_model.predict(X_test)) ,
               'brfpre' : np.exp(ranger.predict(X_test)),
               'cridge' : np.exp(ridge_model.predict(X_test.astype(float))) } ).describe()

### LASSO - Not a good result

In [0]:
%%script false 

from sklearn.linear_model import LassoCV

lasso_model = LassoCV(alphas=[1e-3, 1e-2, 1e-1, 1], cv = 5).fit(X_train.astype(float), y_train.astype(float))
np.sqrt( -cross_val_score(lasso_model, X_train.astype(float), y_train.astype(float), cv = 5, scoring = "neg_mean_squared_error") )
np.sqrt(mean_squared_error(np.exp(y_test.astype(float)), np.exp(lasso_model.predict(X_test.astype(float)))))

### Neural Network - worst

In [0]:
%%script false

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense

def baseline_model():
    model = Sequential()
    model.add(Dense(100, input_dim=12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
  
#model.fit(X_train, y_train)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=5, verbose = 0, validation_split = 0.20)))
pipeline = Pipeline(estimators)
#estimator = KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=5, verbose=0)
# np.sqrt( -cross_val_score(pipeline, X_train.astype(float), y_train.astype(float), cv = 5, scoring = "neg_mean_squared_error") )